
# Executive summary
You can use Stan to write a statistical model and easily compile it to efficient C++ and native code, and cause it to be linked in to a running Python (or R) process while giving you access to not just the obvious HMC sampling methods but also the underying posterior density function $\pi(y \, | \theta)$ and its gradient under `log_prob` and `grad_log_prob`, respectively. This document shows how to do that in Python.

# Context
On the [Stan](http://mc-stan.org) team, we get a lot of questions from people looking to use Stan for algorithm development. Stan is opinionated software and believes that models are mostly separate from inference mechanisms, so most of the people looking to do algorithm development with us buy into that; they're looking to perhaps add some mechanism for discrete parameters or maybe add another type of approximation alternative to [ADVI](https://arxiv.org/abs/1506.03431) for a given statistical model. 

I claim that this version is actually pretty easy to do in practice, but we haven't written anything up about it because the Stan team values robustness and generality first and foremost, and we're eagerly awaiting something that beats the No U-Turn Sampler (which has since been improved in our source code from the [original paper](https://arxiv.org/abs/1111.4246)). There are a few of us who would like to support the broader research community in using Stan for algorithm development, even if we do not include any of these algorithms with the distribution of Stan. This document should illustrate how this can work.

In [6]:
import pystan
import numpy as np

In [85]:
# Construct an arbitrary model
model_code = """
data {
  int numObs;
  int numGroups;
  matrix[numObs, numGroups] group;
  vector[numObs] y;
}
parameters {
  vector[numGroups] beta;
  real<lower=0> sigma;
}
model {
  beta ~ normal(0, 20);
  sigma ~ normal(0, 5);
  y ~ normal(group * beta, sigma);
}
"""
model = pystan.StanModel(model_code=model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_390a0ec538f0cb4f0a4483ba147e0080 NOW.


In [108]:
# fake data
def gen_fake_data(N=100, K=3):
    group_ = np.random.choice(K, N)
    # one-hot encode in matrix
    group = np.zeros((N, K))
    group[np.arange(N), group_] = 1
    beta = np.random.normal(0, 20, K)
    sigma = np.abs(np.random.normal(0, 5))
    print("sigma: {}".format(sigma))
    y = np.random.normal(group.dot(beta), sigma)
    return dict(group=group, y=y, numObs=N, numGroups=K, beta=beta)
fd = gen_fake_data()
fd

sigma: 9.157885217081992


{'group': array([[0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
 

In [109]:
# You must first run one of the methods that gives you a fit object:
fit = model.sampling(data=fd)
fit

Inference for Stan model: anon_model_390a0ec538f0cb4f0a4483ba147e0080.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

          mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
beta[1]  -24.4    0.02   1.45 -27.25 -25.37 -24.42 -23.46 -21.52   5131    1.0
beta[2] -23.44    0.02   1.45 -26.26 -24.43 -23.46 -22.46 -20.59   4251    1.0
beta[3] -20.77    0.03   1.74 -24.19 -21.93 -20.79 -19.66 -17.22   4088    1.0
sigma     8.81    0.01   0.63   7.68   8.36   8.79   9.22  10.14   3649    1.0
lp__    -269.8    0.03   1.47 -273.5 -270.5 -269.4 -268.7 -268.0   1796    1.0

Samples were drawn using NUTS at Sat Oct 20 22:47:52 2018.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).

In [39]:
# This then exposes a log prob and grad_log_prob method:
list(filter(lambda x: "prob" in x, dir(fit)))

['grad_log_prob', 'log_prob']

In [143]:
help(fit.log_prob)

Help on built-in function log_prob:

log_prob(...) method of stanfit4anon_model_390a0ec538f0cb4f0a4483ba147e0080_913984840570262847.StanFit4Model instance
    Expose the log_prob of the model to stan_fit so user can call
    this function.
    
    Parameters
    ----------
    upar : array
        The real parameters on the unconstrained space.
    adjust_transform : bool
        Whether we add the term due to the transform from constrained
        space to unconstrained space implicitly done in Stan.
    
    Note
    ----
    In Stan, the parameters need be defined with their supports. For
    example, for a variance parameter, we must define it on the positive
    real line. But inside Stan's sampler, all parameters defined on the
    constrained space are transformed to unconstrained space, so the log
    density function need be adjusted (i.e., adding the log of the absolute
    value of the Jacobian determinant).  With the transformation, Stan's
    samplers work on the unconstr

In [144]:
fit.log_prob(upar=np.arange(4)) # adjust_transform seems to default to True

-386.70221104212527

In [145]:
# We can pass parameter values to `log_prob` in the order we defined them,
# which is shown in `flatnames`
print(fit.flatnames)
fit.log_prob(upar=[-50, 40, 32, 2.8])

['beta[1]', 'beta[2]', 'beta[3]', 'sigma']


-764.6526305472523

# Expectation Maximization
[Expectation maximization](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm) can be thought of as a generalization of [k-means](https://en.wikipedia.org/wiki/K-means_clustering) with discrete cluster assignments to models that have additional parameters (that might be dependent on the cluster assigments). Bob Carpenter has [a good write-up](https://discourse.mc-stan.org/t/tutorial-on-monte-carlo-em-and-variants-for-mml-and-mmap/5173); I'll try to summarize the algorithm briefly here.

The algorithm is traditionally explained as broken up into two steps, one for calculating the expectation of the cluster assignments (or other latent parameters) and using those to generate the assignments, and another for maximizing the values of the other parameters given those assignments. One way to accomplish this with Stan is to provide two Stan models, one for each step. I'll call these `expectation.stan` and `maximization.stan` and use Markov Chain Monte Carlo to calculate the integral in the expectation step, and our BFGS optimizer for the maximization step. The model we already defined can be used as is for the maximization step


In [105]:
maxi_model = model

In [116]:
expectation_model = pystan.StanModel(model_code="""
data {
  int numObs;
  int numGroups;
  row_vector[numGroups] beta;
  vector[numObs] y;
}
parameters {
  simplex[numGroups] group[numObs];
  real<lower=0> sigma;
}
model {
  vector[numObs] mu;
  for (n in 1:numObs)
    mu[n] = beta * group[n];
  beta ~ normal(0, 20);
  sigma ~ normal(0, 5);
  y ~ normal(mu, sigma);
}
""")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_5fb00b74056ca004b9d6262d4b3f4b40 NOW.


In [117]:
efit = expectation_model.sampling(fd)

In [137]:
g = efit.extract()['group']
np.mean(g, axis=0) # shitty group assignments

array([[0.34287063, 0.33352272, 0.32360666],
       [0.38534671, 0.33009014, 0.28456314],
       [0.32387381, 0.33703026, 0.33909593],
       [0.40500117, 0.32694373, 0.2680551 ],
       [0.30960678, 0.33668936, 0.35370385],
       [0.38028181, 0.32616916, 0.29354904],
       [0.34636906, 0.33375806, 0.31987287],
       [0.33912141, 0.33499342, 0.32588517],
       [0.31187386, 0.33640507, 0.35172107],
       [0.31823332, 0.33544074, 0.34632594],
       [0.365601  , 0.33248204, 0.30191696],
       [0.28854845, 0.33530351, 0.37614805],
       [0.33981265, 0.33450326, 0.32568408],
       [0.35817954, 0.33082139, 0.31099908],
       [0.3564163 , 0.33486351, 0.30872019],
       [0.36999923, 0.32687867, 0.3031221 ],
       [0.38540445, 0.3306891 , 0.28390645],
       [0.30736623, 0.33507612, 0.35755765],
       [0.34348607, 0.33116681, 0.32534712],
       [0.29724348, 0.34231639, 0.36044012],
       [0.29648312, 0.33735369, 0.36616319],
       [0.32303331, 0.33817898, 0.33878772],
       [0.

In [ ]:
def em_algo(maxi_model, expectation_model, data, num_iter=10):
    print("Original beta: {}".format(data['beta']))
    #Random initialization for parameters
    data['beta'] = np.random.normal(0, 20, len(data['beta']))
    for i in range(num_iter):
        print("Iteration {}; beta: {}".format(i, data['beta']))
        # E step
        efit = expectation_model.sampling(data)
        data['group'] = np.mean(efit.extract()['group'], axis=0)
        
        # M step
        mfit = maxi_model.optimizing(data)
        data['beta'] = mfit['beta']
em_algo(maxi_model, expectation_model, fd)

Original beta: [-116.43297767   72.04383948   38.9385973 ]
Iteration 0; beta: [41.25547549 -1.77402583 48.43987292]
Iteration 1; beta: [  34.72164799 -112.12691211   46.61467111]
